# Convergence of exponential approximation

Fitting $e^{-d}$ on $(0,2)$ with sums of Gaussians using differential evolution with least squares followed by Newton refinement.

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path('..') / 'src'))

import numpy as np
import matplotlib.pyplot as plt

from kl_decomposition import gauss_legendre_rule, fit_exp_sum, newton_with_line_search
from kl_decomposition.kernel_fit import _prepare_numpy_funcs

In [ ]:
def l2_error(f, g, x, w):
    return np.sqrt(np.sum(w * (f(x) - g(x))**2))

x, w = gauss_legendre_rule(0.0, 2.0, 1000)
f = lambda t: np.exp(-t)

errors = []
a_prev = None
b_prev = None

for N in range(1, 11):
    if b_prev is None:
        init_means = np.ones(N)
    else:
        init_means = np.hstack([b_prev, b_prev[-1]**2 / b_prev[-2]])[:N]
    init_sigmas = np.ones_like(init_means)
    a_ls, b_ls, _ = fit_exp_sum(N, x, w, f, method='de_ls', compiled=False,
                                max_gen=100, pop_size=20,
                                de_mean=init_means, de_sigma=init_sigmas)
    target = f(x)
    obj, grad, hess = _prepare_numpy_funcs(x, target, w, N, newton=True)
    params0 = np.concatenate([a_ls, np.log(b_ls)])
    params_opt, _ = newton_with_line_search(params0, obj, grad, hess,
                                           max_iter=100, compiled=False)
    a_prev = np.exp(params_opt[:N])
    b_prev = np.exp(params_opt[N:])
    def approx(t, a=a_prev, b=b_prev):
        return np.sum(a[:, None] * np.exp(-b[:, None] * t[None, :]**2), axis=0)
    err = l2_error(f, approx, x, w)
    errors.append(err)

plt.figure(figsize=(6,4))
plt.semilogy(range(1, 11), errors, marker='o')
plt.xlabel('N')
plt.ylabel('L2 error')
plt.title('Convergence of de_ls + Newton fit')
plt.grid(True)
plt.tight_layout()